In [ ]:
!pip install transformers datasets tokenizers
from datasets import load_dataset
from transformers import AutoTokenizer, MBartForConditionalGeneration, Trainer, TrainingArguments, MBart50TokenizerFast

In [7]:
dataset = load_dataset("SKNahin/bengali-transliteration-data")

train_dataset, val_dataset = dataset["train"].train_test_split(test_size=0.2).values()

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX", tgt_lang="bn_IN")

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples['rm'], max_length=128, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['bn'], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

print(model.config)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()